更新的DAM实现了更大的板块：  
![](过曝光+欠曝光同时处理.png)

上图网络结构如下：

过曝光和欠曝光图像分别进入我们已有的DAM模型  
处理后分别于原图相乘再相加，得到可以直接进入下一阶段，即特征增强的图像  

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

class DualAttentionModule(nn.Module):
    def __init__(self, in_channels):
        super(DualAttentionModule, self).__init__()

        # 输入图像的初始处理
        self.initial_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.initial_relu = nn.LeakyReLU(0.1)
        
        # 像素注意力模块
        self.pa_conv1 = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.pa_relu1 = nn.LeakyReLU(0.1)
        self.pa_bn1 = nn.BatchNorm2d(in_channels // 8)
        self.pa_conv2 = nn.Conv2d(in_channels // 8, 1, kernel_size=1)
        self.pa_sigmoid = nn.Sigmoid()

        # 通道注意力模块
        self.ca_gap = nn.AdaptiveAvgPool2d(1)
        self.ca_conv1 = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.ca_relu1 = nn.LeakyReLU(0.1)
        self.ca_conv2 = nn.Conv2d(in_channels // 8, in_channels, kernel_size=1)
        self.ca_sigmoid = nn.Sigmoid()

        # 特征增强模块
        self.fe_conv = nn.Conv2d(in_channels * 2, in_channels, kernel_size=3, padding=1)
        self.fe_tanh = nn.Tanh()

    def forward(self, x):
        # 初始处理
        x2 = self.initial_conv(x)
        x2 = self.initial_relu(x2)
        
        # 像素注意力
        pa = self.pa_conv1(x2)
        pa = self.pa_relu1(pa)
        pa = self.pa_bn1(pa)
        pa = self.pa_conv2(pa)
        pa = self.pa_sigmoid(pa)
        pa_out = x2 * pa
        
        # 通道注意力
        ca = self.ca_gap(x2)
        ca = self.ca_conv1(ca)
        ca = self.ca_relu1(ca)
        ca = self.ca_conv2(ca)
        ca = self.ca_sigmoid(ca)
        ca_out = x2 * ca
        
        # 拼接PA和CA的输出
        concat = torch.cat([pa_out, ca_out], dim=1)
        
        # 最后两层
        fe = self.fe_conv(concat)
        fe = self.fe_tanh(fe)
        
        # 输出
        out = fe + x + x + x2
        
        return out

class HDRNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(HDRNet, self).__init__()
        self.dam = DualAttentionModule(in_channels)
        self.final_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x1, x2):
        # 欠曝光（记作1，处理后记作3）
        x1_dam = self.dam(x1)
        # 过曝光（记作2，处理后记作4）
        x2_dam = self.dam(x2)

        # 1和3相乘记作5
        x3 = x1 * x1_dam

        # 2和4相乘记作6
        x4 = x2 * x2_dam

        # 5和6相加
        out = x3 + x4

        # 最终卷积
        out = self.final_conv(out)
        
        return out

# 测试
def test_model_with_images(image_path1, image_path2, model):
    # 预处理
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # 加载图像
    image1 = Image.open(image_path1).convert('RGB')
    image2 = Image.open(image_path2).convert('RGB')
    input_image1 = transform(image1).unsqueeze(0)  # 添加批次维度
    input_image2 = transform(image2).unsqueeze(0)  # 添加批次维度

    # 将图像输入模型
    model.eval()  # 设置为评估模式
    with torch.no_grad():
        output_image = model(input_image1, input_image2)

    # 反归一化并转换为PIL图像
    output_image = output_image.squeeze(0).detach().cpu()
    output_image = (output_image * 0.5 + 0.5).clamp(0, 1)  # 反归一化
    output_image = transforms.ToPILImage()(output_image)

    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    axs[0].imshow(image1)
    axs[0].set_title('Underexposed Image')
    axs[0].axis('off')
    axs[1].imshow(image2)
    axs[1].set_title('Overexposed Image')
    axs[1].axis('off')
    axs[2].imshow(output_image)
    axs[2].set_title('Output Image')
    axs[2].axis('off')
    plt.show()

model = HDRNet(in_channels=3, out_channels=3)

# 测试图片路径
image_path1 = '欠.jpg'  # 欠曝光
image_path2 = '过.jpg'  # 过曝光

# 测试
test_model_with_images(image_path1, image_path2, model)
